# Simulation of Germanium Segregation During Zone Refining with Vibration Monitoring

## Abstract

This project presents a fully software-based simulation framework for modeling
Germanium impurity segregation during zone refining using the Scheil–Pfann
equation, combined with virtual vibration monitoring of the refining process.
The material purification model predicts impurity redistribution along the rod,
while a mechanical mass–spring–damper model simulates process-induced vibrations.
Virtual accelerometers and frequency-domain analysis using FFT are employed to
analyze the dynamic response under varying process parameters. Monte Carlo
simulations are used to account for process uncertainty. The framework
demonstrates how material-level phenomena can be linked to structural dynamics
and sensor-based diagnostics, providing a unified approach relevant to both
materials research and vibration-based testing and monitoring.


In [ ]:
# Germanium Segregation & Vibration Monitoring Simulation - Interactive Notebook
# Author: Zarmina Shah

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

# ----------------------------
# Scheil-Pfann Equation Function
# ----------------------------
def scheil_pfann(C0, k, L):
    """
    Calculate Germanium concentration along the rod
    """
    return C0 * (1 - L)**(k - 1)

# ----------------------------
# Simulation Function
# ----------------------------
def simulate_germanium_vibration(C0=0.02, k=0.3, MC_runs=5, vib_amp=5.0):
    # Rod positions
    N_points = 500
    L = np.linspace(0, 1, N_points)
    C_profile = scheil_pfann(C0, k, L)

    # Monte Carlo variations
    C_profiles_MC = []
    for _ in range(MC_runs):
        noise = np.random.normal(0, 0.001, N_points)
        C_profiles_MC.append(C_profile + noise)
    C_profiles_MC = np.array(C_profiles_MC)

    # ----------------------------
    # Vibration Simulation (Mass-Spring-Damper)
    # ----------------------------
    m = 2.0
    k_vib = 1000
    c = 2.0
    dt = 0.001
    T = 2
    N_time = int(T/dt)
    t = np.linspace(0, T, N_time)
    
    # Periodic force from moving heater
    F = vib_amp * np.sin(2 * np.pi * 3 * t)
    
    x = np.zeros(N_time)
    v = np.zeros(N_time)
    a = np.zeros(N_time)
    
    for i in range(1, N_time):
        a[i] = (F[i] - c*v[i-1] - k_vib*x[i-1]) / m
        v[i] = v[i-1] + a[i]*dt
        x[i] = x[i-1] + v[i]*dt
    
    # Virtual accelerometer
    acc_noise_std = 0.05
    acc_sensor = a + np.random.normal(0, acc_noise_std, N_time)
    
    # FFT
    yf = fft(acc_sensor)
    xf = fftfreq(N_time, dt)[:N_time//2]
    
    # ----------------------------
    # Visualization
    # ----------------------------
    plt.figure(figsize=(15, 12))
    
    # Germanium Segregation
    plt.subplot(3,1,1)
    plt.plot(L, C_profile, label="Nominal")
    for i in range(MC_runs):
        plt.plot(L, C_profiles_MC[i], linestyle='--', alpha=0.5)
    plt.xlabel("Normalized Rod Length")
    plt.ylabel("Ge Fraction")
    plt.title("Germanium Segregation Profile (Scheil-Pfann)")
    plt.legend()
    plt.grid(True)
    
    # Vibration Time Series
    plt.subplot(3,1,2)
    plt.plot(t, acc_sensor)
    plt.title("Virtual Accelerometer Signal During Zone Refining")
    plt.xlabel("Time (s)")
    plt.ylabel("Acceleration (m/s²)")
    plt.grid(True)
    
    # FFT Spectrum
    plt.subplot(3,1,3)
    plt.plot(xf, 2.0/N_time * np.abs(yf[0:N_time//2]))
    plt.title("Frequency Spectrum of Vibration Signal")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print("Simulation completed for C0 =", C0, ", k =", k, ", MC runs =", MC_runs, ", Vib Amplitude =", vib_amp)

# ----------------------------
# Interactive Sliders
# ----------------------------
interactive_plot = interactive(
    simulate_germanium_vibration,
    C0=widgets.FloatSlider(min=0.01, max=0.05, step=0.005, value=0.02, description='Initial Ge'),
    k=widgets.FloatSlider(min=0.1, max=0.5, step=0.05, value=0.3, description='Seg. Coef k'),
    MC_runs=widgets.IntSlider(min=1, max=10, step=1, value=5, description='MC Runs'),
    vib_amp=widgets.FloatSlider(min=1.0, max=10.0, step=0.5, value=5.0, description='Vib Amplitude')
)

display(interactive_plot)


interactive(children=(FloatSlider(value=0.02, description='Initial Ge', max=0.05, min=0.01, step=0.005), Float…

## Results and Discussion

The simulated Germanium segregation profiles show the expected impurity
accumulation toward the end of the rod, consistent with predictions from the
Scheil–Pfann model. Variations introduced through Monte Carlo simulations
illustrate the sensitivity of impurity distribution to process fluctuations,
such as changes in segregation coefficient and initial concentration.

The vibration simulation reveals periodic acceleration responses induced by the
moving heater during zone refining. Time-domain signals captured by virtual
accelerometers exhibit characteristic oscillatory behavior, while frequency-
domain analysis using FFT highlights dominant excitation frequencies. Changes in
excitation amplitude directly affect vibration magnitude and spectral content,
demonstrating the usefulness of frequency analysis for process monitoring and
anomaly detection.

By combining material segregation modeling with vibration analysis, the
framework illustrates how thermal and mechanical effects during zone refining
can be monitored using sensor-based diagnostics. This approach is directly
relevant to industrial testing, process validation, and structural monitoring
applications.


## Impurity Segregation Model (Scheil–Pfann)

The redistribution of impurities during zone refining is described by the
Scheil–Pfann equation, assuming complete mixing in the molten zone and
negligible diffusion in the solid phase:

\[
C_s(x) = C_0 (1 - x)^{k - 1}
\]

where:

- \( C_s(x) \) is the impurity concentration in the solid
- \( C_0 \) is the initial impurity concentration
- \( k \) is the equilibrium segregation coefficient
- \( x \) is the normalized solidified fraction along the rod \((0 \le x \le 1)\)

This equation is commonly used to model impurity profiles in Germanium
during zone refining processes.


## Thermal Gradient During Zone Refining

The temperature distribution along the rod during zone refining can be
approximated by a one-dimensional thermal gradient:

\[
\frac{dT}{dx} = G
\]

where:

- \( T \) is the temperature
- \( x \) is the position along the rod
- \( G \) is the thermal gradient imposed by the moving heater

Thermal gradients influence both impurity redistribution and induced
mechanical stresses in the material.

## Mechanical Vibration Model

The mechanical response of the rod during zone refining is modeled using a
single-degree-of-freedom mass–spring–damper system:

\[
m \ddot{x}(t) + c \dot{x}(t) + k x(t) = F(t)
\]

where:

- \( m \) is the effective mass of the rod segment
- \( c \) is the damping coefficient
- \( k \) is the stiffness
- \( x(t) \) is the displacement
- \( F(t) \) is the external excitation force induced by the moving heater


## Periodic Excitation Force

The excitation force generated by the moving heater is modeled as a
harmonic function:

\[
F(t) = A \sin(2\pi f t)
\]

where:

- \( A \) is the excitation amplitude
- \( f \) is the excitation frequency
- \( t \) is time

This formulation allows frequency-domain analysis using FFT to identify
dominant vibration modes.

## Frequency-Domain Analysis (FFT)

The acceleration signal measured by virtual sensors is transformed into
the frequency domain using the Fast Fourier Transform (FFT):

\[
X(f) = \sum_{n=0}^{N-1} x(n)\, e^{-j2\pi fn/N}
\]

This enables identification of dominant vibration frequencies and
potential anomalies during the refining process.
